In [ ]:
! pip install langchain
! pip install langchain_openai
! pip install langchain_community
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos

In [ ]:
import json
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "example.env" 
config = dotenv_values(env_name)

openai.api_type = config['openai_type']
# Azure OpenAI connection details
openai_endpoint = config['openai_endpoint']
openai_key = config['openai_key']
openai_version = config['openai_version']
openai_embeddings_deployment = config['openai_embeddings_deployment']
openai_embeddings_model = config['openai_embeddings_model']
openai_embeddings_dimensions = int(config['openai_embeddings_dimensions'])
openai_completions_deployment = config['openai_completions_deployment']
openai_completions_model = config['openai_completions_model']
cosmos_vector_property = config['cosmos_vector_property_name']

In [ ]:
# Connection string
mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ResturantChain"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
#mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["Contoso", "Contoso West", "Users"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

In [ ]:
db.command({
  'createIndexes': 'FoodCollection',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "Embedding": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-hnsw',
        'm': 16,
        'efConstruction': 40,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

In [ ]:
import json 

data_file = open(file="./data/results.json", mode="r") 
data = json.load(data_file)
data_file.close()
vcore_collection = db['FoodCollection']
result = vcore_collection.insert_many(data)

print(f"Number of data points added: {len(result.inserted_ids)} in {vcore_collection.name}")

In [34]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = openai_embeddings_deployment,
    api_key= openai_key,
    azure_endpoint= openai_endpoint,
    model= openai_embeddings_model,
    dimensions= openai_embeddings_dimensions,
)

In [36]:
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
import gradio as gr

# Define the food ordering template
order_prompt_template = """
Hello! You can ask me for food recommendations or customize your order according to your dietary preferences.

{chat_history},

Question: {question}
If you don't know what you want yet, feel free to ask for recommendations!
"""

# Create a Langchain chatbot prompt template
chatbot_prompt = ChatPromptTemplate(
    template=order_prompt_template,
    messages=[],
    input_variables=["question", "chat_history"]
)

def prepare_food_ordering_system(azure_endpoint, openai_key, cosmos_conn, azure_openai_embeddings):
    # Initialize the LangChain LLM for chat
    llm = ChatOpenAI(
        azure_endpoint=azure_endpoint,
        api_key=openai_key,
        cache=True,
        n=1
    )
    
    # Setup Azure CosmosDB Vector Search
    vector_search = AzureCosmosDBVectorSearch.from_connection_string(
        connection_string=cosmos_conn,
        namespace="RestaurantChain.FoodCollection",
        text_key="description", 
        embedding = azure_openai_embeddings,
        embedding_key = cosmos_vector_property,
    )
    
    # Setup retrieval chain
    retriever = vector_search.as_retriever(search_type="similarity", search_kwargs={"k": 5, 'score_threshold': 0.2})
    retrieval_chain = create_retrieval_chain(llm, retriever, prompt=chatbot_prompt, return_source_documents=False)

    return retrieval_chain

# Setup Gradio interface for the food ordering system
def setup_gradio_interface(chain):
    def fetch_response(question, chat_history):
        response = chain.invoke({"question": question, "chat_history": chat_history})
        return response['answer'], chat_history + [question + " - " + response['answer']]
    
    with gr.Blocks() as demo_interface:
        chatbot = gr.Chatbot(label="Food Ordering System")
        msg = gr.Textbox(label="Your question")
        chat_history = gr.State([])
        
        msg.submit(fetch_response, inputs=[msg, chat_history], outputs=[chatbot, chat_history])
    
    return demo_interface

# Prepare the chain and setup the demo
food_ordering_system = prepare_food_ordering_system(
    azure_endpoint=openai_endpoint,
    openai_key=openai_key,
    cosmos_conn=mongo_conn,
    azure_openai_embeddings = azure_openai_embeddings
)

# Launch the Gradio interface
food_ordering_demo = setup_gradio_interface(food_ordering_system)
food_ordering_demo.launch()


C:\Users\Khelan Modi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! azure_endpoint is not default parameter.
                azure_endpoint was transferred to model_kwargs.
                Please confirm that azure_endpoint is what you intended.
  warnings.warn(


TypeError: create_retrieval_chain() got an unexpected keyword argument 'prompt'